In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import  DataLoader
from tqdm import tqdm

from utils.networks import DecoderRNN, build_resnet18

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
device

'cuda'

In [22]:
encoder = build_resnet18()
encoder.load_state_dict(torch.load('params_/resnet18_contrastive.pt'))
encoder = encoder.to(device)

In [18]:




        
        
        

linear_eval_model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in tqdm(test_loader, desc="Evaluating test"):
        inputs, labels = inputs.to(device), labels.to(device)
        features, _ = encoder(inputs)
        outputs = linear_eval_model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_accuracy = (correct / total) * 100
    
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(train_loader, desc="Evaluating train"):
            inputs, labels = inputs.to(device), labels.to(device)
            features, _ = encoder(inputs)
            outputs = linear_eval_model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    train_accuracy = (correct / total) * 100

return train_accuracy, test_accuracy


Files already downloaded and verified
Files already downloaded and verified


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 40/40 [00:01<00:00, 20.67it/s]


Training: 100%|██████████| 10/10 [00:01<00:00,  5.47it/s]


In [57]:
linear_evaluation(encoder, 10)

Files already downloaded and verified
Files already downloaded and verified


Evaluating test: 100%|██████████| 10/10 [00:00<00:00, 381.48it/s]


(66.682, 65.78)